In [46]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from carla.data.catalog import CsvCatalog

import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json

seed = 44

tf.compat.v1.random.set_random_seed(seed)
np.random.seed(seed)

In [49]:
# Prep adult dataset

with open('../data/adult_constraints.json', 'r') as f:
    constraints = json.load(f)

continuous = constraints['continuous']
categorical = constraints['categorical']
immutable = constraints['immutable']
columns_order = constraints['features_order']

In [50]:
dataset = CsvCatalog(file_path="../data/adult_prep.csv",
                    continuous=continuous,
                    categorical=categorical,
                    immutables=immutable,
                    target='income')



In [42]:
df = dataset.df_train

X_train = dataset.df_train[columns_order]
Y_train = df['income']
Y_train = pd.get_dummies(Y_train)
Y_train

,<=50K,>50K
9739,1,0
23394,0,1
25744,1,0
2631,0,1
15550,1,0
...,...,...
16955,1,0
25773,0,1
27377,1,0
3491,1,0


In [43]:
df = dataset.df_test
X_test = dataset.df_test[columns_order]
Y_test = df['income']
Y_test = pd.get_dummies(Y_test)
Y_test

,<=50K,>50K
13099,1,0
21918,1,0
23550,1,0
6892,0,1
17909,0,1
...,...,...
14474,1,0
9451,0,1
6348,1,0
3396,1,0


In [44]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((len(X_train.columns),)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.fit(
    X_train.to_numpy(), 
    Y_train.to_numpy(),
    epochs=100,
    batch_size=128,
    validation_data=(X_test.to_numpy(), Y_test.to_numpy()),
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ]
)

Train on 24420 samples, validate on 8141 samples
Epoch 1/100


ValueError: could not convert string to float: 'HS-grad'

In [36]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [38]:
len(X_train.columns)

107

In [39]:
columns_order

['age',
 'fnlwgt',
 'education.num',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'marital.status_Divorced',
 'marital.status_Married-AF-spouse',
 'marital.status_Married-civ-spouse',
 'marital.status_Married-spouse-absent',
 'marital.status_Never-married',
 'marital.status_Separated',
 'marital.status_Widowed',
 'native.country_?',
 'native.country_Cambodia',
 'native.country_Canada',
 'native.country_China',
 'native.country_Columbia',
 'native.country_Cuba',
 'native.country_Dominican-Republic',
 'native.country_Ecuador',
 'native.country_El-Salvador',
 'native.country_England',
 'native.country_France',
 'native.country_Germany',
 'native.country_Greece',
 'native.country_Guatemala',
 'native.country_Haiti',
 'native.country_Holand-Netherlands',
 'native.country_Honduras',
 'native.country_Hong',
 'native.country_Hungary',
 'native.country_India',
 'native.country_Iran',
 'native.country_Ireland',
 'native.country_Italy',
 'native.country_Jamaica',
 'native.country_Japan',